In [1]:
import torch
import os
from nets.mae.models_mae import mae_vit_base_patch16, mae_vit_large_patch16
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)
from help_func import  print_var_detail

device: cuda:0


In [2]:
# ====== model and training ======
net = mae_vit_base_patch16()
optimizer = torch.optim.AdamW(net.parameters(), lr=1.5e-4)

In [3]:
INPUT_SIZE = 224
BATCH_SIZE = 2

input = torch.randn((BATCH_SIZE, 3, INPUT_SIZE, INPUT_SIZE))
print_var_detail(input)

 is a  <class 'torch.Tensor'> with shape torch.Size([2, 3, 224, 224]) max:  tensor(4.3060) min:  tensor(-5.0016)


In [4]:
net

MaskedAutoencoderViT(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  (decoder_embed): Linear(in_features=768, out_features=512, bias=

In [5]:
# 
for param in net.patch_embed.parameters():
    param.requires_grad = False
for param in net.blocks.parameters():
    param.requires_grad = False
for param in net.norm.parameters():
    param.requires_grad = False

In [5]:
for param in net.parameters():
    param.requires_grad = False

In [6]:
has_freeze_param = 0
for param in net.blocks.parameters():
    if param.requires_grad == False:
        has_freeze_param += 1
print(has_freeze_param)

144


In [7]:
net.train()

MaskedAutoencoderViT(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  (decoder_embed): Linear(in_features=768, out_features=512, bias=

In [9]:
x, mask, ids_restore = net.forward_encoder(input, 0)
# remove cls token
x = x[:, 1:, :]
print_var_detail(x)

 is a  <class 'torch.Tensor'> with shape torch.Size([2, 196, 768]) max:  tensor(4.5289, grad_fn=<MaxBackward1>) min:  tensor(-4.3109, grad_fn=<MinBackward1>)


In [40]:
# pred = net.forward_decoder(x, ids_restore)

In [44]:
x = net.unpatchify(x)

In [45]:
print_var_detail(x)

 is a  <class 'torch.Tensor'> with shape torch.Size([2, 3, 224, 224]) max:  tensor(4.2414, grad_fn=<MaxBackward1>) min:  tensor(-4.3223, grad_fn=<MinBackward1>)
